# Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from scipy.spatial import cKDTree
from pyproj import Transformer
from netCDF4 import Dataset
from urllib import request
from datetime import date
import pandas as pd
import numpy as np
import geocoder
import rasterio
import zipfile
import os

# Root directory

In [ ]:
root_directory_path = "./datasets"

if not os.path.exists(root_directory_path):
  os.makedirs(root_directory_path)

In [ ]:
limit_values = 10

# Forest Fire Data

Near real-time (NRT) Moderate Resolution Imaging Spectroradiometer (MODIS) Thermal Anomalies / Fire locations - Collection 61 processed by NASA's Land, Atmosphere Near real-time Capability for EO (LANCE) Fire Information for Resource Management System (FIRMS), using swath products (MOD14/MYD14) rather than the tiled MOD14A1 and MYD14A1 products. The thermal anomalies / active fire represent the center of a 1 km pixel that is flagged by the MODIS MOD14/MYD14 Fire and Thermal Anomalies algorithm (Giglio 2003) as containing one or more fires within the pixel. This is the most basic fire product in which active fires and other thermal anomalies, such as volcanoes, are identified.

For more information [here](https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/mcd14dl-nrt#ed-firms-attributes)

In [ ]:
remote_url = "https://firms.modaps.eosdis.nasa.gov/data/download/DL_FIRE_M-C61_443477.zip"
local_file = f"{root_directory_path}/forest_fire_Colombia.zip"
local_csv_1 = f"{root_directory_path}/forest_fire_Colombia_2002_2023.csv"
request.urlretrieve(remote_url, local_file)

with zipfile.ZipFile(local_file, 'r') as zip_ref:
  zip_ref.extractall(root_directory_path)

os.remove(local_file)
os.remove(f"{root_directory_path}/Readme.txt")
os.remove(f"{root_directory_path}/fire_nrt_M-C61_443477.csv")
os.rename(f"{root_directory_path}/fire_archive_M-C61_443477.csv", local_csv_1)

In [ ]:
remote_url = "https://firms.modaps.eosdis.nasa.gov/data/download/DL_FIRE_SV-C2_446719.zip"
local_file = f"{root_directory_path}/forest_fire_Colombia.zip"
local_csv_2 = f"{root_directory_path}/forest_fire_Colombia_2012_2020.csv"
request.urlretrieve(remote_url, local_file)

with zipfile.ZipFile(local_file, 'r') as zip_ref:
  zip_ref.extractall(root_directory_path)

os.remove(local_file)
os.remove(f"{root_directory_path}/Readme.txt")
os.rename(f"{root_directory_path}/fire_archive_SV-C2_446719.csv", local_csv_2)

In [ ]:
df_ff_2002_2023 = pd.read_csv(local_csv_1, low_memory=False)
df_ff_2012_2020 = pd.read_csv(local_csv_2, low_memory=False)

df_ff_2002_2023['acq_date'] = pd.to_datetime(df_ff_2002_2023['acq_date'])
df_ff_2012_2020['acq_date'] = pd.to_datetime(df_ff_2012_2020['acq_date'])

date_min = df_ff_2012_2020['acq_date'].min()
date_max = df_ff_2012_2020['acq_date'].max()

df_ff_2002_2012 = df_ff_2002_2023[df_ff_2002_2023['acq_date'] < date_min]
df_ff_2020_2023 = df_ff_2002_2023[date_max < df_ff_2002_2023['acq_date']]

df_forest_fire = pd.concat([df_ff_2002_2012, df_ff_2012_2020, df_ff_2020_2023])
os.remove(local_csv_1)
os.remove(local_csv_2)

In [ ]:
df_forest_fire = df_forest_fire.sort_values(by="acq_date").dropna()
df_forest_fire.to_csv(f"{root_directory_path}/forest_fire_Colombia.csv", index=False)
df_forest_fire.head(limit_values)

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,3.6370,-75.1719,306.9,1.0,1.0,2002-07-01,324,Terra,MODIS,69,6.03,293.7,5.4,N,0
13,4.7785,-75.8046,306.0,1.1,1.0,2002-07-01,1543,Terra,MODIS,61,6.03,283.1,3.9,D,0
12,10.2584,-74.0905,333.5,1.1,1.1,2002-07-01,1542,Terra,MODIS,65,6.03,306.1,23.2,D,0
11,10.4153,-74.3193,321.7,1.1,1.1,2002-07-01,1542,Terra,MODIS,68,6.03,304.0,9.6,D,0
9,9.6186,-74.5987,321.3,1.1,1.0,2002-07-01,1542,Terra,MODIS,69,6.03,304.0,9.3,D,0
8,9.8017,-73.0799,319.3,1.3,1.1,2002-07-01,1542,Terra,MODIS,27,6.03,300.5,14.3,D,0
7,9.8085,-73.0564,320.8,1.3,1.1,2002-07-01,1542,Terra,MODIS,37,6.03,301.2,16.6,D,0
10,8.9371,-73.5798,309.9,1.3,1.1,2002-07-01,1542,Terra,MODIS,48,6.03,288.0,7.7,D,0
5,10.2533,-74.0873,323.1,1.1,1.1,2002-07-01,1542,Terra,MODIS,64,6.03,304.6,11.9,D,0
4,10.3046,-73.2843,323.4,1.3,1.1,2002-07-01,1542,Terra,MODIS,66,6.03,305.7,8.8,D,0


In [ ]:
lat_values, lon_values = df_forest_fire['latitude'].to_numpy(), df_forest_fire['longitude'].to_numpy()
lat_lon_values = list(set([(lat, lon) for lat, lon in zip(lat_values, lon_values)]))

In [ ]:
df_forest_fire['year'] = df_forest_fire['acq_date'].astype(str).str.slice(start=0, stop=4).astype(int)
interpolated_values = df_forest_fire[df_forest_fire['year'] <= 2020][['latitude', 'longitude', 'year']].values
extrapolated_values = df_forest_fire[2021 <= df_forest_fire['year']][['latitude', 'longitude', 'year']].values

# NDVI Data

This dataset contains dekadal NDVI indicators computed from NASA's Moderate Resolution Imaging Spectroradiometer (MODIS) collection 6.1 from the Aqua and Terra satellite aggregated by sub-national administrative units.

Included indicators are (for each dekad):

- 10 day NDVI (vim)
- NDVI long term average (vim_lta)
- 10 day NDVI anomaly [%] (viq)

The administrative units used for aggregation are based on WFP data and contain a Pcode reference attributed to each unit. The number of input pixels used to create the aggregates, is provided in the n_pixels column.

More information [here](https://data.humdata.org/dataset/col-ndvi-subnational)

In [ ]:
remote_url = "https://data.humdata.org/dataset/7f2ba5ba-8df1-41cf-ab18-fc1da928a1e5/resource/c06298d9-0d4d-4e40-aecc-abc1da75dc4d/download/col-ndvi-adm2-full.csv"
local_file_ndvi_dataset = f"{root_directory_path}/ndvi_Colombia.csv"
request.urlretrieve(remote_url, local_file_ndvi_dataset)

('./datasets/ndvi_Colombia.csv', <http.client.HTTPMessage at 0x7fac5e4cd3c0>)

In [ ]:
df_ndvi = pd.read_csv(local_file_ndvi_dataset, low_memory=False)
df_ndvi = df_ndvi.drop(df_ndvi.index[0])
df_ndvi['date'] = pd.to_datetime(df_ndvi['date'])
df_ndvi = df_ndvi[df_ndvi['date'] <= pd.to_datetime("2023-12-31")]

In [ ]:
postal_codes = list(set(df_ndvi["ADM2_PCODE"].values.astype('str')))
postal_codes = np.array([postal_code.replace("CO", "") for postal_code in postal_codes]).astype(int)

In [ ]:
remote_url = "https://www.datos.gov.co/api/views/ixig-z8b5/rows.csv?accessType=DOWNLOAD"
postal_codes_path = f"{root_directory_path}/postal_codes.csv"
request.urlretrieve(remote_url, postal_codes_path)

('./datasets/postal_codes.csv', <http.client.HTTPMessage at 0x7fac767f8e50>)

In [ ]:
column_name = "codigo_municipio"
df_postal_codes = pd.read_csv(postal_codes_path)

df_postal_codes[column_name] = df_postal_codes[column_name].replace(',', '').astype(int)
df_postal_codes = df_postal_codes.drop_duplicates(subset=column_name, keep='first')

result = df_postal_codes[df_postal_codes[column_name].isin(postal_codes)][['nombre_departamento', 'nombre_municipio', 'codigo_municipio', 'codigo_postal']]
result.reset_index(drop=True, inplace=True)
result.sort_values(by="codigo_postal")

,nombre_departamento,nombre_municipio,codigo_municipio,codigo_postal
303,ANTIOQUIA,MEDELLIN,5001,50.021
794,ANTIOQUIA,SAN PEDRO DE LOS MILAGROS,5664,51.010
1010,ANTIOQUIA,BARBOSA,5079,51.028
16,ANTIOQUIA,GIRARDOTA,5308,51.037
874,ANTIOQUIA,COPACABANA,5212,51.040
...,...,...,...,...
1028,VAUPES,PAPUNAUA,97777,973.047
883,VICHADA,PUERTO CARREÑO,99001,990.008
419,VICHADA,CUMARIBO,99773,991.001
126,VICHADA,LA PRIMAVERA,99524,992.001


In [ ]:
tam = int(result.size / result.columns.size)
lat_lon_ndvi = np.zeros((tam, 2))

for index, [municipality, department] in enumerate(result[['nombre_municipio', 'nombre_departamento']].values):
  location = geocoder.osm(f"{municipality}, {department}, Colombia").latlng
  if location is None:
    location = geocoder.osm(f"{department}, Colombia").latlng
  lat_lon_ndvi[index] = np.array(location)

In [ ]:
result['latitude'] = lat_lon_ndvi[:, 0]
result['longitude'] = lat_lon_ndvi[:, 1]

result.rename(columns={'codigo_municipio': 'ADM2_PCODE'}, inplace=True)
result['ADM2_PCODE'] = 'CO' + result['ADM2_PCODE'].astype(str)

In [ ]:
merged_df_ndvi = pd.merge(df_ndvi, result[['latitude', 'longitude', 'ADM2_PCODE']], on='ADM2_PCODE', how='left')
merged_df_ndvi = merged_df_ndvi.drop(columns={'adm2_id', 'ADM2_PCODE'})

In [ ]:
values = {key: [] for key in merged_df_ndvi.columns}
merged_df_ndvi = merged_df_ndvi.dropna()

for year in range(2002, 2024):
  # Filtramos los datos
  date_min, date_max = pd.to_datetime(f'{year}-01-01'), pd.to_datetime(f'{year}-12-31')
  df_ndvi_temp = merged_df_ndvi[(date_min <= merged_df_ndvi['date']) & (merged_df_ndvi['date'] <= date_max)]
  df_forest_fire_temp = df_forest_fire[(date_min <= df_forest_fire['acq_date']) & (df_forest_fire['acq_date'] <= date_max)]

  df_forest_fire_temp = df_forest_fire_temp.rename(columns={'acq_date': 'date'})
  df_ndvi_temp.reset_index(drop=True, inplace=True)
  df_forest_fire_temp.reset_index(drop=True, inplace=True)
  init_date = pd.to_datetime(f'{year}-01-01')

  # forest fire values
  lat_values = df_forest_fire_temp['latitude'].values
  lon_values = df_forest_fire_temp['longitude'].values
  time_values = (df_forest_fire_temp['date'] - init_date).dt.days.values

  # ndvi values
  lat = df_ndvi_temp['latitude'].values
  lon = df_ndvi_temp['longitude'].values
  time = (df_ndvi_temp['date'] - init_date).dt.days.values

  points = np.vstack((lat, lon, time)).T
  tree = cKDTree(points)
  query_points = np.vstack((lat_values, lon_values, time_values)).T
  _, indexes = tree.query(query_points)

  for key in ['latitude', 'longitude', 'date']:
    values[key] += list(df_forest_fire_temp[key].values)

  for key in ['n_pixels', 'vim', 'vim_avg', 'viq']:
    values[key] += list(df_ndvi_temp.iloc[indexes][key].values)

  print(f"Año {year} terminado")

Año 2002 terminado
Año 2003 terminado
Año 2004 terminado
Año 2005 terminado
Año 2006 terminado
Año 2007 terminado
Año 2008 terminado
Año 2009 terminado
Año 2010 terminado
Año 2011 terminado
Año 2012 terminado
Año 2013 terminado
Año 2014 terminado
Año 2015 terminado
Año 2016 terminado
Año 2017 terminado
Año 2018 terminado
Año 2019 terminado
Año 2020 terminado
Año 2021 terminado
Año 2022 terminado
Año 2023 terminado


In [ ]:
df_final_ndvi = pd.DataFrame(values).sort_values(by="date").dropna()
df_final_ndvi.to_csv(local_file_ndvi_dataset, index=False)
os.remove(postal_codes_path)

In [ ]:
df_final_ndvi.head(limit_values)

,date,n_pixels,vim,vim_avg,viq,latitude,longitude
0,2002-07-01,28.0,0.6682,0.6317,105.3523,3.6370,-75.1719
13,2002-07-01,6.0,0.8004,0.7816,102.2621,10.3468,-75.3190
12,2002-07-01,7.0,0.7248,0.6983,103.5393,11.0892,-72.6697
11,2002-07-01,24.0,0.7834,0.7974,98.356,10.6032,-73.4690
9,2002-07-01,24.0,0.6592,0.7453,89.171,10.3046,-73.2843
8,2002-07-01,13.0,0.711,0.7702,92.7813,10.2533,-74.0873
7,2002-07-01,18.0,0.7971,0.7946,100.3005,8.9371,-73.5798
10,2002-07-01,13.0,0.711,0.7702,92.7813,10.2548,-74.0975
5,2002-07-01,37.0,0.7156,0.7604,94.4796,9.8017,-73.0799
4,2002-07-01,16.0,0.7413,0.7151,103.4241,9.6186,-74.5987


# Global Climate Data

TerraClimate is a dataset of monthly climate and climatic water balance for global terrestrial surfaces from 1958-2019. These data provide important inputs for ecological and hydrological studies at global scales that require high spatial resolution and time-varying data. All data have monthly temporal resolution and a ~4-km (1/24th degree) spatial resolution. The data cover the period from 1958-2020. We plan to update these data periodically (annually).

More information [here](https://www.climatologylab.org/terraclimate.html)

In [ ]:
def check_latlon_bounds(lat,lon,lat_index,lon_index,lat_target,lon_target):
  #check final indices are in right bounds
  if(lat[lat_index]>lat_target):
      if(lat_index!=0):
          lat_index = lat_index - 1
  if(lat[lat_index]<lat_target):
      if(lat_index!=len(lat)):
          lat_index = lat_index +1
  if(lon[lon_index]>lon_target):
      if(lon_index!=0):
          lon_index = lon_index - 1
  if(lon[lon_index]<lon_target):
      if(lon_index!=len(lon)):
          lon_index = lon_index + 1

  return [lat_index, lon_index]

def get_indexes(data, points):
  data_reshaped = data.filled().reshape(-1, 1)
  tree = cKDTree(data_reshaped)
  query_points = points.to_numpy().reshape(-1, 1)
  _, indexes = tree.query(query_points)

  return indexes

def get_data_country(df_modis, varnames):
  values = {varname: [] for varname in ["date", "lat", "lon"] + varnames}
  df_modis["acq_date"] = pd.to_datetime(df_modis["acq_date"])
  df_modis = df_modis.sort_values(by="acq_date")

  for year in range(2002, 2024):
    print(f"Descargando datos del año {year}: [ ", end="")
    df = df_modis[df_modis["acq_date"] <= pd.to_datetime(f"{year}-12-31")]
    df = df[pd.to_datetime(f"{year}-01-01") <= df["acq_date"]]

    date_values, lat_values, lon_values = df['acq_date'], df['latitude'], df['longitude']
    lat_min, lon_min = lat_values.min(), lon_values.min()
    lat_max, lon_max = lat_values.max(), lon_values.max()

    values['date'] += [str(date_.date()) for date_ in date_values]
    values['lat'] += list(lat_values.values)
    values['lon'] += list(lon_values.values)
    time_values = (date_values - pd.to_datetime("1900-01-01")).dt.days

    for varname in varnames:
      print(f"{varname} ", end="")
      pathname = f"http://thredds.northwestknowledge.net:8080/thredds/dodsC/TERRACLIMATE_ALL/data/TerraClimate_{varname}_{year}.nc"
      filehandle = Dataset(pathname, 'r', format="NETCDF4")

      # subset in space (lat/lon)
      lathandle = filehandle.variables['lat']
      lonhandle = filehandle.variables['lon']
      lat=lathandle[:]
      lon=lonhandle[:]

      # find indices of target lat/lon/day
      lat_index_min = (np.abs(lat-lat_min)).argmin()
      lat_index_max = (np.abs(lat-lat_max)).argmin()
      lon_index_min = (np.abs(lon-lon_min)).argmin()
      lon_index_max = (np.abs(lon-lon_max)).argmin()

      [lat_index_min,lon_index_min] = check_latlon_bounds(lat, lon, lat_index_min, lon_index_min, lat_min, lon_min)
      [lat_index_max,lon_index_max] = check_latlon_bounds(lat, lon, lat_index_max, lon_index_max, lon_max, lon_max)

      if(lat_index_min>lat_index_max):
          lat_index_range = range(lat_index_max, lat_index_min+1)
      else:
          lat_index_range = range(lat_index_min, lat_index_max+1)
      if(lon_index_min>lon_index_max):
          lon_index_range = range(lon_index_max, lon_index_min+1)
      else:
          lon_index_range = range(lon_index_min, lon_index_max+1)

      lat=lat[lat_index_range]
      lon=lon[lon_index_range]

      # subset in time
      timehandle=filehandle.variables['time']
      time=timehandle[:]
      time_min = (date(year,1,1)-date(1900,1,1)).days
      time_max = (date(year,12,31)-date(1900,1,1)).days
      time_index_min = (np.abs(time-time_min)).argmin()
      time_index_max = (np.abs(time-time_max)).argmin()
      time_index_range = range(time_index_min, time_index_max+1)
      time = timehandle[time_index_range]

      # subset data
      datahandle = filehandle.variables[varname]
      data = datahandle[time_index_range, lat_index_range, lon_index_range]

      # Indexes
      time_indexes = get_indexes(time, time_values)
      lat_indexes = get_indexes(lat, lat_values)
      lon_indexes = get_indexes(lon, lon_values)

      values[varname] += list(data[time_indexes, lat_indexes, lon_indexes])
    print("]")
  return values

In [ ]:
varnames = ["ws", "vpd", "vap", "tmin", "tmax", "swe", "srad", "soil", "q", "ppt", "pet", "def", "aet", "PDSI"]
global_climate_path = f"{root_directory_path}/global_climate_Colombia.csv"
values = get_data_country(df_forest_fire, varnames)
df_global_climate = pd.DataFrame(values)
for varname in varnames:
  df_global_climate[varname] = df_global_climate[varname].astype(float, copy=True)

df_global_climate['date'] = pd.to_datetime(df_global_climate['date'])
df_global_climate = df_global_climate.sort_values(by="date").dropna()
df_global_climate.to_csv(global_climate_path, index=False)

Descargando datos del año 2002: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2003: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2004: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2005: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2006: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2007: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2008: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2009: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2010: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2011: [ ws vpd vap tmin tmax swe srad soil q ppt pet def aet PDSI ]
Descargando datos del año 2012: [ ws vpd vap tmin tmax swe s

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:170: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [ ]:
df_global_climate.head(limit_values)

,date,lat,lon,ws,vpd,vap,tmin,tmax,swe,srad,soil,q,ppt,pet,def,aet,PDSI
0,2002-07-01,3.6370,-75.1719,1.23,1.50,2.017,19.66,31.84,0.0,179.0,89.3,2.5,50.3,112.8,23.9,88.9,1.88
13,2002-07-01,9.8085,-73.0564,2.65,0.82,1.987,15.93,28.01,0.0,229.2,229.6,3.2,64.6,130.6,25.5,105.1,-0.94
12,2002-07-01,4.7785,-75.8046,1.09,0.91,2.036,16.09,29.18,0.0,171.7,39.0,4.7,94.7,98.5,3.2,95.3,1.28
11,2002-07-01,10.2584,-74.0905,2.79,1.24,2.937,22.09,35.09,0.0,231.3,140.4,5.3,105.3,154.4,31.3,123.1,1.52
9,2002-07-01,9.6186,-74.5987,2.10,1.13,2.989,22.42,34.54,0.0,218.9,215.2,7.2,144.4,139.6,1.1,138.5,1.54
8,2002-07-01,9.8017,-73.0799,2.65,0.82,1.987,15.93,28.01,0.0,229.2,229.6,3.2,64.6,130.6,25.5,105.1,-0.94
7,2002-07-01,10.2548,-74.0975,2.79,1.24,2.937,22.09,35.09,0.0,231.3,140.4,5.3,105.3,154.4,31.3,123.1,1.52
10,2002-07-01,10.4153,-74.3193,3.08,1.15,3.000,21.94,34.98,0.0,233.6,128.8,5.9,117.2,154.9,28.8,126.1,1.25
5,2002-07-01,10.2533,-74.0873,2.79,1.24,2.937,22.09,35.09,0.0,231.3,140.4,5.3,105.3,154.4,31.3,123.1,1.52
4,2002-07-01,10.3046,-73.2843,2.96,1.61,2.844,23.27,36.18,0.0,227.0,102.5,3.6,71.2,166.2,57.7,108.5,1.17


# Land cover data

The Intergovernmental Panel on Climate Change (IPCC) provides guidance on reporting areal extent and change of land cover and land use, requiring the use of estimators that neither over or underestimate dynamics to the degree possible, and that have known uncertainties. The maps provided by GLAD do not have these properties. However, the maps can be leveraged to facilitate appropriate probability-based statistical methods in deriving statistically valid areas of forest extent and change. Specifically, the maps may be used as a stratifier in targeting forest extent and/or change by a probability sample. The team at GLAD has demonstrated such approaches using the GLAD forest loss data in sample-based area estimation (Tyukavina et al., ERL, 2018, Turubanova et al., ERL, 2019, and Potapov et al., RSE, 2019, among others).

More information [here](https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/download.html)

Legend [here](https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/legend.xlsx)

In [ ]:
range_values = [(20, 80), (10, 80), (10, 70), ('00', 80), ('00', 70)]
for year in range(2000, 2021, 5):
  for N, W in range_values:
    remote_url = f"https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/{year}/{N}N_0{W}W.tif"
    land_cover_path = f"{root_directory_path}/land_cover_Colombia_{year}_{N}N_0{W}W.tif"
    request.urlretrieve(remote_url, land_cover_path)
    print(f"Descargado: {land_cover_path}")

Descargado: ./datasets/land_cover_Colombia_2000_20N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2000_10N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2000_10N_070W.tif
Descargado: ./datasets/land_cover_Colombia_2000_00N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2000_00N_070W.tif
Descargado: ./datasets/land_cover_Colombia_2005_20N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2005_10N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2005_10N_070W.tif
Descargado: ./datasets/land_cover_Colombia_2005_00N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2005_00N_070W.tif
Descargado: ./datasets/land_cover_Colombia_2010_20N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2010_10N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2010_10N_070W.tif
Descargado: ./datasets/land_cover_Colombia_2010_00N_080W.tif
Descargado: ./datasets/land_cover_Colombia_2010_00N_070W.tif
Descargado: ./datasets/land_cover_Colombia_2015_20N_080W.tif
Descargado: ./datasets/l

In [ ]:
def get_land_cover(lat_lon_array, land_cover_path):
  with rasterio.open(land_cover_path) as src:
    transform = src.transform
    tif_crs = src.crs
    transformer = Transformer.from_crs("epsg:4326", tif_crs, always_xy=True)
    lon_values, lat_values = lat_lon_array[:,1], lat_lon_array[:,0]
    x_coords, y_coords = transformer.transform(lon_values, lat_values)
    row, col = rasterio.transform.rowcol(transform, x_coords, y_coords)
    values = src.read(1)[row, col]

  return lat_values, lon_values, values

In [ ]:
lat_lon_values_20N_80W = list(filter(lambda lat_lon: 10 < lat_lon[0] <= 20 and -80 <= lat_lon[1] < -70, lat_lon_values))
lat_lon_values_10N_80W = list(filter(lambda lat_lon: 0 < lat_lon[0] <= 10 and -80 <= lat_lon[1] < -70, lat_lon_values))
lat_lon_values_10N_70W = list(filter(lambda lat_lon: 0 < lat_lon[0] <= 10 and -70 <= lat_lon[1] < -60, lat_lon_values))
lat_lon_values_00N_80W = list(filter(lambda lat_lon: -10 < lat_lon[0] <= 0 and -80 <= lat_lon[1] < -70, lat_lon_values))
lat_lon_values_00N_70W = list(filter(lambda lat_lon: -10 < lat_lon[0] <= 0 and -70 <= lat_lon[1] < -60, lat_lon_values))

In [ ]:
def save_land_cover_values(values, name):
  land_cover_values = {'lat': [], 'lon': [], 'year': [], 'land_cover': []}
  lat_lon_array = np.array(values)
  for year in range(2000, 2021, 5):
    land_cover_path = f"{root_directory_path}/land_cover_Colombia_{year}_{name}.tif"
    lat_values, lon_values, result = get_land_cover(lat_lon_array, land_cover_path)

    land_cover_values['lat'] += list(lat_values)
    land_cover_values['lon'] += list(lon_values)
    land_cover_values['year'] += list(np.full(len(lat_values), year))
    land_cover_values['land_cover'] += list(result)

    os.remove(f"{root_directory_path}/land_cover_Colombia_{year}_{name}.tif")
    print(f"{land_cover_path} descargado")

  df = pd.DataFrame(land_cover_values)
  df.to_csv(f"{root_directory_path}/land_cover_Colombia_{name}.csv", index=False)
  print("Datos descargados")

In [ ]:
save_land_cover_values(lat_lon_values_00N_70W, "00N_070W")
save_land_cover_values(lat_lon_values_00N_80W, "00N_080W")
save_land_cover_values(lat_lon_values_10N_70W, "10N_070W")
save_land_cover_values(lat_lon_values_20N_80W, "20N_080W")
save_land_cover_values(lat_lon_values_10N_80W, "10N_080W")

./datasets/land_cover_Colombia_2000_00N_070W.tif descargado
./datasets/land_cover_Colombia_2005_00N_070W.tif descargado
./datasets/land_cover_Colombia_2010_00N_070W.tif descargado
./datasets/land_cover_Colombia_2015_00N_070W.tif descargado
./datasets/land_cover_Colombia_2020_00N_070W.tif descargado
Datos descargados
./datasets/land_cover_Colombia_2000_00N_080W.tif descargado
./datasets/land_cover_Colombia_2005_00N_080W.tif descargado
./datasets/land_cover_Colombia_2010_00N_080W.tif descargado
./datasets/land_cover_Colombia_2015_00N_080W.tif descargado
./datasets/land_cover_Colombia_2020_00N_080W.tif descargado
Datos descargados
./datasets/land_cover_Colombia_2000_10N_070W.tif descargado
./datasets/land_cover_Colombia_2005_10N_070W.tif descargado
./datasets/land_cover_Colombia_2010_10N_070W.tif descargado
./datasets/land_cover_Colombia_2015_10N_070W.tif descargado
./datasets/land_cover_Colombia_2020_10N_070W.tif descargado
Datos descargados
./datasets/land_cover_Colombia_2000_20N_080W.t

In [ ]:
df_land_cover = pd.DataFrame()
for N, W in range_values:
  land_cover_csv_path = f"{root_directory_path}/land_cover_Colombia_{N}N_0{W}W.csv"
  df_tlc = pd.read_csv(land_cover_csv_path)
  df_land_cover = pd.concat([df_land_cover, df_tlc])
  os.remove(f"{root_directory_path}/land_cover_Colombia_{N}N_0{W}W.csv")

df_land_cover = df_land_cover.sort_values(by="year").dropna()

In [ ]:
X = df_land_cover[['lat', 'lon', 'year']].values
y = df_land_cover['land_cover'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = DecisionTreeRegressor(max_depth=30, random_state=42)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(score)

0.790096417239057


In [ ]:
land_covers_interpolated = model.predict(interpolated_values)
land_covers_extrapolated = model.predict(extrapolated_values)

In [ ]:
df_land_cover_predicted = pd.DataFrame({
    'latitude': np.append(interpolated_values[:, 0], extrapolated_values[:, 0]),
    'longitude': np.append(interpolated_values[:, 1], extrapolated_values[:, 1]),
    'year': np.append(interpolated_values[:, 2], extrapolated_values[:, 2]).astype(int),
    'land_cover': np.append(land_covers_interpolated, land_covers_extrapolated).astype(int)
})
df_land_cover_predicted = df_land_cover_predicted.sort_values(by="year")
df_land_cover_predicted.to_csv(f"{root_directory_path}/land_cover_Colombia.csv", index=False)

In [ ]:
df_land_cover_predicted.head(limit_values)

,latitude,longitude,year,land_cover
0,3.6370,-75.1719,2002.0,26
5815,3.2009,-75.4335,2002.0,24
5814,2.7053,-72.1792,2002.0,24
5813,2.9420,-72.2366,2002.0,24
5812,5.8592,-69.1002,2002.0,23
5811,4.9122,-74.3044,2002.0,32
5810,3.4949,-72.5849,2002.0,23
5809,3.5262,-72.5802,2002.0,24
5808,5.6320,-70.0255,2002.0,124
5807,5.6414,-70.0242,2002.0,123


In [ ]:
remote_url = f"https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/legend.xlsx"
land_cover_legend_path = f"{root_directory_path}/land_cover_legend_Colombia.xlsx"
request.urlretrieve(remote_url, land_cover_legend_path)

In [ ]:
df_land_cover_legend = pd.read_excel(land_cover_legend_path)
df_land_cover_legend.rename(columns={'Unnamed: 2': 'class'}, inplace=True)

def set_values(column1, column2, indexes, nan_indexes=[]):
    for start_index, end_index in indexes:
        total = end_index - start_index + 1
        df_land_cover_legend.loc[np.linspace(start_index, end_index, total), column1] = df_land_cover_legend.at[start_index, column2]

    for nan_index in nan_indexes:
        df_land_cover_legend.at[nan_index, column1] = np.NAN

# Same column
set_values('General class', 'General class', [(0, 96), (100, 196), (200, 207)], [97, 197, 208, 242, 245, 251, 255])
set_values('class', 'class', [(0, 1), (2, 18), (19, 24), (25, 48), (100, 101), (102, 118), (119, 124), (125, 148)], [49, 149])

# Other column
set_values('class', 'General class', [(200, 207), (241, 241), (244, 244), (250, 250), (254, 254)])
set_values('Sub-class', 'General class', [(241, 241), (244, 244), (250, 250), (254, 254)])

df_land_cover_legend.drop(columns={"Color code"}, inplace=True)
df_land_cover_legend.to_csv(f"{root_directory_path}/land_cover_legend_Colombia.csv", index=False)
os.remove(land_cover_legend_path)

In [ ]:
df_land_cover_legend.head(limit_values)

,Map value,General class,class,Sub-class
0,0,Terra Firma,True desert,3% short vegetation cover
1,1,Terra Firma,True desert,7% short vegetation cover
2,2,Terra Firma,Semi-arid,11% short vegetation cover
3,3,Terra Firma,Semi-arid,15% short vegetation cover
4,4,Terra Firma,Semi-arid,19% short vegetation cover
5,5,Terra Firma,Semi-arid,23% short vegetation cover
6,6,Terra Firma,Semi-arid,27% short vegetation cover
7,7,Terra Firma,Semi-arid,31% short vegetation cover
8,8,Terra Firma,Semi-arid,35% short vegetation cover
9,9,Terra Firma,Semi-arid,39% short vegetation cover


# Population Density Data

Estimated population density per grid-cell. The dataset is available to download in Geotiff and ASCII XYZ format at a resolution of 30 arc (approximately 1km at the equator). The projection is Geographic Coordinate System, WGS84. The units are number of people per square kilometre based on country totals adjusted to match the corresponding official United Nations population estimates that have been prepared by the Population Division of the Department of Economic and Social Affairs of the United Nations Secretariat (2019 Revision of World Population Prospects). The mapping approach is Random Forest-based dasymetric redistribution.

More information [here](https://hub.worldpop.org/geodata/summary?id=45716)

In [ ]:
for year in range(2002, 2021):
  remote_url = f"https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km/{year}/COL/col_pd_{year}_1km_ASCII_XYZ.zip"
  local_file = f"{root_directory_path}/population_density_Colombia_{year}.zip"
  request.urlretrieve(remote_url, local_file)

  with zipfile.ZipFile(local_file, 'r') as zip_ref:
    zip_ref.extractall(root_directory_path)
  os.remove(local_file)

In [ ]:
for year in range(2002, 2021):
  pd_path = f"{root_directory_path}/col_pd_{year}_1km_ASCII_XYZ.csv"
  if os.path.exists(pd_path):
    df_pd = pd.read_csv(pd_path)
    df_pd.rename(columns={'X': 'longitude', 'Y': 'latitude', 'Z': 'population_density'}, inplace=True)

    # Filtramos por fecha
    date_min, date_max = pd.to_datetime(f"{year}"), pd.to_datetime(f"{year + 1}")
    df_ff = df_forest_fire[(date_min <= df_forest_fire['acq_date']) & (df_forest_fire['acq_date'] <= date_max)]
    lat_values, lon_values = df_ff['latitude'], df_ff['longitude']

    # Minimos
    lat_min, lat_max = lat_values.min(), lat_values.max()
    lon_min, lon_max = lon_values.min(), lon_values.max()

    # Filtramos las latitudes
    df_pd.sort_values(by="latitude")
    df_pd = df_pd[lat_min <= df_pd['latitude']]
    df_pd = df_pd[df_pd['latitude'] <= lat_max]

    # Filtramos las longitudes
    df_pd.sort_values(by="longitude")
    df_pd = df_pd[lon_min <= df_pd['longitude']]
    df_pd = df_pd[df_pd['longitude'] <= lon_max]

    # Establecemos valores
    df_pd.reset_index(drop=True, inplace=True)
    lat, lon = df_pd['latitude'].to_numpy(), df_pd['longitude'].to_numpy()

    # Hallamos los valores
    points = np.vstack((lat, lon)).T
    tree = cKDTree(points)
    query_points = np.vstack((lat_values, lon_values)).T
    _, indices = tree.query(query_points)
    population_density_values = df_pd.iloc[indices]['population_density'].to_numpy()

    # Guardamos los datos
    df_population_density = pd.DataFrame({'latitude': lat_values, 'longitude': lon_values,
                                'year': np.full(len(lat_values), year), 'population_density': population_density_values})
    df_population_density.to_csv(f"{root_directory_path}/population_density_Colombia_{year}.csv", index=False)
    os.remove(pd_path)
  print(f"Año {year} terminado")

In [ ]:
# Unimos los datos
df_population_density = pd.DataFrame()
for year in range(2002, 2021):
  df_ppd = pd.read_csv(f"{root_directory_path}/population_density_Colombia_{year}.csv")
  df_population_density = pd.concat([df_population_density, df_ppd])
  os.remove(f"{root_directory_path}/population_density_Colombia_{year}.csv")

df_population_density = df_population_density.sort_values(by="year").dropna()

In [ ]:
X = df_population_density[['latitude', 'longitude', 'year']].values  # Características: año, latitud y longitud
y = df_population_density['population_density'].values             # Densidad de población como variable dependiente

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = DecisionTreeRegressor(max_depth=20, random_state=42)
regressor.fit(X_train, y_train)

score = regressor.score(X_test, y_test)
print(score)

0.9286478134261494


In [ ]:
densities_predicted = regressor.predict(extrapolated_values)

In [ ]:
df_pd_predicted = pd.DataFrame({
    'latitude': extrapolated_values[:, 0],
    'longitude': extrapolated_values[:, 1],
    'year': extrapolated_values[:, 2].astype(int),
    'population_density': densities_predicted
})
df_pd_predicted = pd.concat([df_population_density, df_pd_predicted]).sort_values(by="year")
df_pd_predicted.to_csv(f"{root_directory_path}/population_density_Colombia.csv", index=False)

In [ ]:
df_pd_predicted.head(limit_values)

,latitude,longitude,year,population_density
0,3.6370,-75.1719,2002,19.789324
14,6.5047,-74.6853,2002,9.887000
4406,11.0854,-72.6807,2002,209.546951
1,4.7785,-75.8046,2002,443.014130
2,10.2584,-74.0905,2002,13.135686
3,10.4153,-74.3193,2002,9.273766
4,9.6186,-74.5987,2002,15.801455
5,9.8017,-73.0799,2002,3.144978
6,9.8085,-73.0564,2002,2.751253
7,8.9371,-73.5798,2002,14.648424
